<a href="https://colab.research.google.com/github/ZhyShe/18ma573ZhiyuanShe/blob/master/src/hw4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective is to get familiar with model calibration.
- download call today's option prices of your favrite stock with maturity $T_1$ and several near the money strikes $(K_1, \ldots, K_n)$.
- compute IV with your bsm price engine for each option price. You are going to use USD Libor 3-month for your interest rate input. 
- plot a figure of strike v.s. IV. Do you find any volatility smile? 
- calibrate bsm volatility for the option prices, denote it by $\hat \sigma$. You may use any error function for the calibration purpose.
- reproduce option prices using your price engine with calibrated volatility, then compare how close they are to the market prices.


In [0]:
!git clone https://github.com/ZhyShe/18ma573ZhiyuanShe

Cloning into '18ma573ZhiyuanShe'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 184 (delta 63), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (184/184), 265.32 KiB | 10.61 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [0]:
cd /content/18ma573ZhiyuanShe/src/

/content/18ma573ZhiyuanShe/src


In [0]:
ls hw3_py.py

hw3_py.py


In [0]:
import numpy as np
import matplotlib.pyplot as plt 
from hw3_py import GBM, Option
import scipy.optimize as so
import csv


In [0]:
f=open('datacall.csv')
reader=csv.reader(f)
data=[row in reader]
f.close

FileNotFoundError: ignored

In [0]:
L_rate=2.73263/100.0
today = np.datetime64('2019-02-04')
maturity = np.datetime64('2019-06-21')
time_to_maturity_days = maturity - today
T_1 = time_to_maturity_days/np.timedelta64(1,'D')/365

Init_state=171.25
Drift_ratio=L_rate
maturity=T_1

gbm = GBM(
    init_state = Init_state, #market data
    drift_ratio = Drift_ratio, #market data
    vol_ratio = .1 #initial guess
)

def Clist(Odata):
  Olist=[]
  Slist=[]
  for row in Odata:
    strike, market_price=row
    Slist.apend(strike)
    Olist.append(Option(Otype=1,Strike=strike, Maturity=T_1,Market_Price=market_price))
    return Olist, Slist

def targetfunction(sigma, gbm, option):
  gbm.vol_ratio=sigma
  return np.abs(GBM.bsm_price(gbm, option)-option.Market_Price)

def implied_volatility(gbm, Olist):
  Vlist=[]
  init_vol=.1
  for option in Olist:
    Vlist.append(float(so.fmin(targetfunction, init_vol, args=(gbm, option), disp=0)[0]))
  return Vlist

Olist, Slist=Clist(Odata)
Vlist=implied_volatility(gbm, Olist)
plt.plot(Slist, Vlist)
